In [ ]:
# データの永続化
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

In [ ]:
# ソースからのHuggingface Transformersのインストール
!git clone https://github.com/huggingface/transformers -b v4.4.2
!pip install -e transformers
!pip install fugashi[unidic-lite]
!pip install ipadic

In [ ]:
# メニュー「ランタイム → ランタイムを再起動」で「Google Colab」を再起動

# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

In [ ]:
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

In [ ]:
%%time

# ファインチューニングの実行
!python ./transformers/examples/text-classification/run_glue.py \
    --model_name_or_path=cl-tohoku/bert-base-japanese-whole-word-masking \
    --do_train \
    --do_eval \
    --max_seq_length=128 \
    --per_device_train_batch_size=32 \
    --use_fast_tokenizer=False \
    --learning_rate=2e-5 \
    --num_train_epochs=10 \
    --train_file=train.csv \
    --validation_file=dev.csv \
    --output_dir=output/ \
    --overwrite_output_dir \
    --logging_steps=100

In [ ]:
# 学習状況の確認
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
import torch
from transformers import BertJapaneseTokenizer, AutoModelForSequenceClassification

# トークナイザーとモデルの準備
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking') 
model = AutoModelForSequenceClassification.from_pretrained('output/')    

# テキスト
text = "新型Mac発売！"

# テキストをテンソルに変換
input = tokenizer.encode(text, return_tensors='pt')

# 推論
labels = ['IT', 'スポーツ', '映画']
model.eval()
with torch.no_grad():
    outputs = model(input)[0]
    print(labels[torch.argmax(outputs)])

In [ ]:
%%time

# GLUEのSST-2によるファインチューニングの実行
!python ./transformers/examples/text-classification/run_glue.py \
    --model_name_or_path bert-base-uncased \
    --task_name=sst2 \
    --do_train \
    --do_eval \
    --max_seq_length=128 \
    --per_device_train_batch_size=32 \
    --learning_rate=2e-5 \
    --num_train_epochs=3 \
    --output_dir=output/ \
    --overwrite_output_dir